In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
random.seed(9999999)

In [2]:
# Load all of the saved files
train_feat = np.loadtxt('train_feat.gzip', dtype = 'float')
gallery_feat = np.loadtxt('gallery_feat.gzip', dtype = 'float')
query_feat = np.loadtxt('query_feat.gzip', dtype = 'float')
val_feat = np.loadtxt('val_feat.gzip', dtype = 'float')
train_noval_feat = np.loadtxt('train_noval_feat.gzip', dtype = 'float')
train_labels = np.loadtxt('train_labels.gzip', dtype = 'uint16')
gallery_labels = np.loadtxt('gallery_labels.gzip', dtype = 'uint16')
query_labels = np.loadtxt('query_labels.gzip', dtype = 'uint16')
val_labels = np.loadtxt('val_labels.gzip', dtype = 'uint16')
train_noval_labels = np.loadtxt('train_noval_labels.gzip', dtype = 'uint16')
#train_filelist = np.loadtxt('train_filelist.gzip', dtype = 'uint16')
#gallery_filelist = np.loadtxt('gallery_filelist.gzip', dtype = 'uint16')
#query_filelist = np.loadtxt('query_filelist.gzip', dtype = 'uint16')
#val_filelist = np.loadtxt('val_filelist.gzip', dtype = 'uint16')
#train_noval_filelist = np.loadtxt('train_noval_filelist.gzip', dtype = 'uint16')
train_camId = np.loadtxt('train_camId.gzip', dtype = 'uint16') # uint8
gallery_camId = np.loadtxt('gallery_camId.gzip', dtype = 'uint16')
query_camId = np.loadtxt('query_camId.gzip', dtype = 'uint16')
val_camId = np.loadtxt('val_camId.gzip', dtype = 'uint16')
train_noval_camId = np.loadtxt('train_noval_camId.gzip', dtype = 'uint16')

In [3]:
print(train_labels.shape)
print(train_feat.shape)
train_l = np.reshape(train_labels, (7368,1))
print(train_l.shape)


train_matrix = np.append(train_feat, np.reshape(train_labels, (7368,1)), axis=1)



(7368,)
(7368, 2048)
(7368, 1)


In [4]:
def _pairwise_distances(embeddings, squared=False):
    """Compute the 2D matrix of distances between all the embeddings.

    Args:
        embeddings: tensor of shape (batch_size, embed_dim)
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        pairwise_distances: tensor of shape (batch_size, batch_size)
    """
    # Get the dot product between all embeddings
    # shape (batch_size, batch_size)
    dot_product = tf.matmul(embeddings, tf.transpose(embeddings))

    # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
    # This also provides more numerical stability (the diagonal of the result will be exactly 0).
    # shape (batch_size,)
    square_norm = tf.diag_part(dot_product)

    # Compute the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    # shape (batch_size, batch_size)
    distances = tf.expand_dims(square_norm, 0) - 2.0 * dot_product + tf.expand_dims(square_norm, 1)

    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances = tf.maximum(distances, 0.0)

    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = tf.to_float(tf.equal(distances, 0.0))
        distances = distances + mask * 1e-16

        distances = tf.sqrt(distances)

        # Correct the epsilon added: set the distances on the mask to be exactly 0.0
        distances = distances * (1.0 - mask)

    return distances

def _get_anchor_positive_triplet_mask(labels):
    """Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check that i and j are distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    # Combine the two masks
    mask = tf.logical_and(indices_not_equal, labels_equal)

    return mask


def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    mask = tf.logical_not(labels_equal)

    return mask


def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.
    A triplet (i, j, k) is valid if:
        - i, j, k are distinct
        - labels[i] == labels[j] and labels[i] != labels[k]
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    """
    # Check that i, j and k are distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)
    i_not_equal_j = tf.expand_dims(indices_not_equal, 2)
    i_not_equal_k = tf.expand_dims(indices_not_equal, 1)
    j_not_equal_k = tf.expand_dims(indices_not_equal, 0)

    distinct_indices = tf.logical_and(tf.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)


    # Check if labels[i] == labels[j] and labels[i] != labels[k]
    label_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    i_equal_j = tf.expand_dims(label_equal, 2)
    i_equal_k = tf.expand_dims(label_equal, 1)

    valid_labels = tf.logical_and(i_equal_j, tf.logical_not(i_equal_k))

    # Combine the two masks
    mask = tf.logical_and(distinct_indices, valid_labels)

    return mask

In [5]:
def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.

    For each anchor, we get the hardest positive and hardest negative to form a triplet.

    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels)
    mask_anchor_positive = tf.to_float(mask_anchor_positive)

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = tf.multiply(mask_anchor_positive, pairwise_dist)

    # shape (batch_size, 1)
    hardest_positive_dist = tf.reduce_max(anchor_positive_dist, axis=1, keepdims=True)

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels)
    mask_anchor_negative = tf.to_float(mask_anchor_negative)

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist = tf.reduce_max(pairwise_dist, axis=1, keepdims=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist = tf.reduce_min(anchor_negative_dist, axis=1, keepdims=True)

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    triplet_loss = tf.maximum(hardest_positive_dist - hardest_negative_dist + margin, 0.0)

    # Get final mean triplet loss
    triplet_loss = tf.reduce_mean(triplet_loss)

    return triplet_loss

In [6]:
n_input = 2048   # input layer (28x28 pixels)
n_hidden1 = 1024 # 1st hidden layer
n_hidden2 = 512 # 2nd hidden layer
n_hidden3 = 256 #3rd hidden layer
n_output = 256   # output layer (0-9 digits)

learning_rate = 1e-4
n_iterations = 10000
batch_person_size = 35
batch_image_size = 4
batch_size = batch_person_size * batch_image_size
dropout = 0.5

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None])
keep_prob = tf.placeholder(tf.float32) 

layer_1 = tf.layers.dense(X, n_hidden1, tf.nn.relu)
layer_norm_1 = tf.layers.batch_normalization(layer_1)
layer_drop_1 = tf.nn.dropout(layer_norm_1, keep_prob)
layer_2 = tf.layers.dense(layer_drop_1, n_hidden2, tf.nn.relu)
layer_norm_2 = tf.layers.batch_normalization(layer_2)
layer_3 = tf.layers.dense(layer_norm_2, n_hidden3, tf.nn.relu)
layer_drop_3 = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.layers.dense(layer_norm_1, n_output)

hard_triplet_loss = batch_hard_triplet_loss(Y, output_layer, 100)
train_step = tf.train.AdamOptimizer(1e-4).minimize(hard_triplet_loss)

features = output_layer

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

unique_id_train = np.unique(train_labels) #find all unique training IDs

# train on mini batches
for i in range(n_iterations):
    batch_x = np.zeros((batch_size,2048))
    batch_y = np.zeros(batch_size)
    np.random.shuffle(train_matrix)
    unique_id_NN = unique_id_train[np.array(random.sample(range(unique_id_train.shape[0]),batch_person_size))] # pick 18 random unique training IDs, random.sample does not work on numpy array
    for j in range(batch_person_size):
        bool_idx_NN = train_matrix[:,2048] == unique_id_NN[j]
        batch_x[j*batch_image_size:(j+1)*batch_image_size,:] = train_matrix[bool_idx_NN,:2048][:batch_image_size,:]
        batch_y[j*batch_image_size:(j+1)*batch_image_size] = train_matrix[bool_idx_NN,2048][:batch_image_size]
        
        
    
    batch_x, batch_y = train_matrix[:batch_size,:2048],train_matrix[:batch_size,2048]
    sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})

    # print loss and accuracy (per minibatch)
    if i%100==0:
        #minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
        #print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))
        print("Iteration", str(i))
        
#test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
#print("\nAccuracy on test set:", test_accuracy)



Iteration 0
Iteration 100
Iteration 200
Iteration 300
Iteration 400
Iteration 500
Iteration 600
Iteration 700
Iteration 800
Iteration 900
Iteration 1000
Iteration 1100
Iteration 1200
Iteration 1300
Iteration 1400
Iteration 1500
Iteration 1600
Iteration 1700
Iteration 1800
Iteration 1900
Iteration 2000
Iteration 2100
Iteration 2200
Iteration 2300
Iteration 2400
Iteration 2500
Iteration 2600
Iteration 2700
Iteration 2800
Iteration 2900
Iteration 3000
Iteration 3100
Iteration 3200
Iteration 3300
Iteration 3400
Iteration 3500
Iteration 3600
Iteration 3700
Iteration 3800
Iteration 3900
Iteration 4000
Iteration 4100
Iteration 4200
Iteration 4300
Iteration 4400
Iteration 4500
Iteration 4600
Iteration 4700
Iteration 4800
Iteration 4900
Iteration 5000
Iteration 5100
Iteration 5200
Iteration 5300
Iteration 5400
Iteration 5500
Iteration 5600
Iteration 5700
Iteration 5800
Iteration 5900
Iteration 6000
Iteration 6100
Iteration 6200
Iteration 6300
Iteration 6400
Iteration 6500
Iteration 6600
Iterati

In [7]:
gallery_feat_Neural = sess.run(features, feed_dict={X: gallery_feat, Y: gallery_labels, keep_prob:1.0})
query_feat_Neural = sess.run(features, feed_dict={X: query_feat, Y: query_labels, keep_prob:1.0})

In [8]:
feat_dict = {}
feat_dict["Neural Network"] = (query_feat_Neural, gallery_feat_Neural)

print(query_feat_Neural.shape)

(1400, 256)


In [9]:
#Function that takes as input an index that refers to a query image
# and returns the gallery without the unnecessary pictures in it
# Remove the pictures from the gallery that have the same label and are taken by the same camera as out QUERY[index]
# picture in the query array
def get_removed_gallery (index, t_gallery_feat):
    selected_indices = np.logical_not(np.logical_and(gallery_camId == query_camId[index], gallery_labels == query_labels[index]))
    return t_gallery_feat[selected_indices, :], gallery_labels[selected_indices], gallery_camId[selected_indices]

In [10]:
for key, (t_query_feat, t_gallery_feat) in feat_dict.items():
    # k nearest neighbours implementation
    k = 10
    top_k_accuracies_rank = np.zeros((query_labels.shape[0],k))
    top_k_accuracies_classical = np.zeros((query_labels.shape[0],k))
    top_k_accuracies_recall = np.zeros((query_labels.shape[0],k))

    for i in range (0, query_labels.shape[0], 1):
        gallery_feat_removed, gallery_labels_removed, gallery_camId_removed = get_removed_gallery (i, t_gallery_feat)
        nbrs = NearestNeighbors(n_neighbors=k).fit(gallery_feat_removed)
        distances, indices = nbrs.kneighbors(t_query_feat[i:i+1, :])
        print(gallery_labels_removed[indices], query_labels[i])
        print("camIds:", gallery_camId_removed[indices], query_camId[i])
        is_same_label = (query_labels[i] == gallery_labels_removed[indices])
    
        for j in range (0, k, 1):
            top_k_accuracies_rank[i,j] = np.sum(is_same_label[0, :(j+1)]) != 0
            top_k_accuracies_classical[i,j]=np.sum(is_same_label[0, :(j+1)])/(j+1)
            top_k_accuracies_recall[i,j] = np.sum(is_same_label[0, :(j+1)])/np.sum(gallery_labels_removed==query_labels[i])

    average_precisions = np.zeros((query_labels.shape))
    for j in range (0, query_labels.shape[0], 1):
        recall = top_k_accuracies_recall[j, :]
        precision = top_k_accuracies_classical[j, :]
        index = (recall).argsort()[::-1] # argsort returns the indices that would sort an array (in this case the vector recall)
        recall = recall[index]
        precision = precision[index]
        recall_range = np.arange(0, 1.1, 0.1)
        precision_range = np.zeros((recall_range.shape))
        for i in range (0, recall_range.shape[0], 1):    
            if (precision[recall>=recall_range[i]].size != 0):
                precision_range[i] = np.max(precision[recall>=recall_range[i]])
            else:     
                precision_range[i] = 0
        average_precisions[j] = np.mean(precision_range)
    
    print (key, "Mean average precision:", average_precisions.mean())

[[1368 1426 1426   14   14 1450 1450 1368 1426 1426]] 3
camIds: [[1 2 2 1 1 1 1 1 2 2]] 1
[[ 517   51  235  404  267  235  235   51   99 1426]] 3
camIds: [[1 1 1 2 2 1 2 2 2 2]] 2
[[  6  77  77   6   6 678   6  77 739 678]] 6
camIds: [[2 1 1 2 2 2 2 1 2 2]] 1
[[  6  72  72   6 678 678   6 678   6 678]] 6
camIds: [[1 1 1 1 2 2 1 1 1 1]] 2
[[  7   7   7   7 441 382 441 422 441  22]] 7
camIds: [[2 2 2 2 1 1 2 1 1 1]] 1
[[  7   7   7 426  22 319 319 426 319   7]] 7
camIds: [[1 1 1 1 1 2 2 1 2 1]] 2
[[ 390  738  738  721  721   55 1405  738   11   11]] 11
camIds: [[2 1 1 2 2 1 1 2 2 2]] 1
[[ 721  721  721  721  738   32  738 1434  738  738]] 11
camIds: [[2 2 1 2 2 2 1 1 1 2]] 2
[[ 14  14 616  14   3  51  51  51 650  51]] 14
camIds: [[2 2 2 2 1 2 2 1 1 2]] 1
[[ 14  14  14  14  47  35 291  47  35  47]] 14
camIds: [[1 1 1 1 2 1 1 2 1 1]] 2
[[ 17  17  17  17 715  32 115  32  32 115]] 17
camIds: [[2 2 2 2 2 1 2 1 2 2]] 1
[[ 17  17  17  17 715 715 115 715 715 115]] 17
camIds: [[1 1 1 1 2 1 1 2 1 

[[727 727 727 174 303 303 303 174 303   7]] 119
camIds: [[2 2 2 2 2 2 2 2 1 2]] 2
[[   3  404  404  979 1389  742 1098  699    3  650]] 120
camIds: [[2 2 2 2 2 1 2 1 2 2]] 1
[[646 120 646 120 646 646 120 267 725 807]] 120
camIds: [[2 1 2 1 1 2 1 1 2 2]] 2
[[ 121  121  121  121 1187 1321  558  829  558  227]] 121
camIds: [[2 2 2 2 2 1 1 1 1 2]] 1
[[ 121  121  121  653  558  653  558  653  829 1187]] 121
camIds: [[1 1 1 2 1 2 1 2 1 2]] 2
[[123 123 123 123 450 843 903 824 903 450]] 123
camIds: [[2 2 2 2 1 2 1 2 2 1]] 1
[[123 123 381 859 859 499 699 381 843 381]] 123
camIds: [[1 1 2 1 1 1 1 2 2 2]] 2
[[127 127 127 127  86 494  86  86 494 732]] 127
camIds: [[2 2 2 2 2 1 2 2 2 1]] 1
[[127 127 127 127 732 494 732 494 494 732]] 127
camIds: [[1 1 1 1 1 1 2 2 1 2]] 2
[[128 128 552 823 128 823 163 552 552 552]] 128
camIds: [[2 2 1 1 2 1 2 1 1 1]] 1
[[128 128 129 128 241 241 390 128 241 129]] 128
camIds: [[1 1 2 1 1 1 1 1 1 2]] 2
[[175 360 548 548 164 175 705 424 424 705]] 129
camIds: [[2 2 1 1 2 

[[213 926 213 967 294 817 294 255 926 822]] 213
camIds: [[2 2 2 2 2 2 2 1 2 1]] 1
[[205 205 751 755 135 755 751 294 205 163]] 213
camIds: [[2 2 1 2 2 2 1 1 2 2]] 2
[[423 216 217 423 217 360 557 205 555 157]] 214
camIds: [[2 1 2 2 2 2 2 1 2 1]] 1
[[214 214 214 214 360 205 205 205 157 708]] 214
camIds: [[1 1 1 1 2 1 1 1 1 1]] 2
[[ 310  555  310  555  555  669 1371  310  310  669]] 216
camIds: [[1 2 1 2 2 1 2 2 1 1]] 1
[[349 216 216 424 304 232 424 792 232 163]] 216
camIds: [[2 1 1 2 1 1 2 1 1 2]] 2
[[217 217 217 217 361 423 669 557 566 566]] 217
camIds: [[2 2 2 2 2 2 1 2 2 2]] 1
[[217 217 217 557 423 566 216 566 566 423]] 217
camIds: [[1 1 1 2 2 2 1 2 1 2]] 2
[[ 570 1365  103  471 1365  218  140  471  452  218]] 218
camIds: [[1 1 2 2 1 2 2 2 1 2]] 1
[[229 229 229 229 229 229 229 550 229 730]] 218
camIds: [[2 2 2 2 1 1 1 2 1 2]] 2
[[219 487 519 487 487 487 519 592 592 454]] 219
camIds: [[2 1 1 1 1 1 1 1 1 2]] 1
[[519 219 519 519 219 219 487 219 487 487]] 219
camIds: [[1 1 1 2 1 1 1 1 1 1]

[[763 316 763 401 401 401 247 211 763 219]] 316
camIds: [[1 1 1 1 1 1 2 1 1 2]] 2
[[220 763 374 652 763 316 325 374 374 448]] 317
camIds: [[2 2 1 2 2 1 1 2 2 1]] 1
[[ 652  317  652  317  593  652 1407  674  652  674]] 317
camIds: [[1 1 1 1 1 1 1 2 2 2]] 2
[[ 319  319  319   22  319 1451 1451  303  303   22]] 319
camIds: [[2 2 2 1 2 2 2 1 1 1]] 1
[[ 319  319  319  319  303  303  303  303 1451   22]] 319
camIds: [[1 1 1 1 2 1 2 2 2 1]] 2
[[694 599 903 599 450 903 450 694 599 694]] 320
camIds: [[1 2 2 1 1 2 1 1 2 1]] 1
[[ 660  720  660  720  802  720  720  301 1293  660]] 320
camIds: [[1 1 2 1 1 1 2 2 1 2]] 2
[[ 321  321  321  321  541  592 1372  575  826  541]] 321
camIds: [[2 2 2 2 1 2 1 1 2 1]] 1
[[ 321  321  321  321 1421   96  220  826   96 1372]] 321
camIds: [[1 1 1 1 2 1 1 2 1 1]] 2
[[ 345  345  322  345  345 1354  322  322  322 1354]] 322
camIds: [[1 1 2 1 1 2 2 2 2 2]] 1
[[322 322 322 266 345 736 736 322 772 266]] 322
camIds: [[1 1 1 1 1 1 1 1 1 1]] 2
[[325 666 824 358 358 325 49

[[   3    3    3  267 1426  979  650  471 1426 1426]] 404
camIds: [[2 2 2 2 1 2 1 2 1 2]] 2
[[ 33 394  33  33  33  70 394  33   6  70]] 408
camIds: [[2 1 2 2 1 1 1 1 1 1]] 1
[[378 242  70 408 435  33 242  70 394 378]] 408
camIds: [[2 1 2 1 1 2 2 1 2 2]] 2
[[390 640 640 640 640 390 521 409 409 304]] 409
camIds: [[2 1 1 1 1 2 1 2 2 1]] 1
[[304 818 292 278 292 296 292 278 304 278]] 409
camIds: [[1 1 2 1 2 2 2 1 1 1]] 2
[[410 673 673 423 569 146 410 673 205 205]] 410
camIds: [[2 1 1 2 2 1 2 1 1 1]] 1
[[410 423 360 423 164 360 410 164 146 423]] 410
camIds: [[1 2 2 2 2 1 1 2 1 2]] 2
[[367 367 413 382 413 382 471 413 382 763]] 413
camIds: [[1 1 2 2 2 2 1 2 2 1]] 1
[[742 742 789 742 413 789 742 742 742 789]] 413
camIds: [[2 2 1 1 1 1 2 2 1 1]] 2
[[418 418 418 547 266 345 547 337 418  72]] 418
camIds: [[2 2 2 1 1 2 1 1 2 1]] 1
[[418 345 418 337 418 418  60  60 547 337]] 418
camIds: [[1 2 1 1 1 1 2 2 2 1]] 2
[[175 175 175 335 705 175 133 496 175 150]] 419
camIds: [[2 2 2 2 1 2 2 2 1 1]] 1
[[115 

...

[[ 971  971  988  938 1041 1122  970  971 1122 1041]] 988
camIds: [[1 1 2 1 1 1 1 1 1 1]] 1
[[ 988 1063  971  971 1063  988 1122  988  947  988]] 988
camIds: [[1 1 1 1 1 1 1 1 2 1]] 2
[[ 989 1148 1148  989 1148  968 1037  968  968 1037]] 989
camIds: [[2 2 2 2 2 2 2 2 2 1]] 1
[[ 989  923  923  923  923 1153  923  989  923  922]] 989
camIds: [[1 2 1 1 1 2 2 1 2 1]] 2
[[ 991 1007 1037 1142 1127 1179  968 1257  991 1007]] 991
camIds: [[2 1 1 1 1 2 1 1 2 1]] 1
[[1043 1221 1221 1043 1043 1043 1043 1221 1221 1043]] 991
camIds: [[2 2 2 1 2 2 1 1 2 1]] 2
[[ 998 1234  998 1230 1011 1234 1011 1259  998 1230]] 998
camIds: [[2 1 2 1 1 1 1 2 2 1]] 1
[[ 998  998  998 1230 1259 1123 1259 1011 1011  913]] 998
camIds: [[1 1 1 1 2 1 2 1 1 1]] 2
[[ 999  999 1042 1068 1224  999  999 1068 1128 1407]] 999
camIds: [[2 2 1 2 2 2 2 2 2 2]] 1
[[ 999  999  930 1042  999 1224  999 1148 1148  930]] 999
camIds: [[1 1 1 1 1 2 1 1 2 2]] 2
[[1002 1002 1243 1243 1155 1155 1155  983  983  983]] 1002
camIds: [[2 2 1 1 1 1

[[1209 1093 1209 1093 1032 1209 1093 1209 1270 1032]] 1093
camIds: [[2 2 2 2 2 2 2 1 2 1]] 1
[[1209 1093 1209 1032 1142 1209 1209 1093 1242 1032]] 1093
camIds: [[2 1 2 2 1 2 1 1 1 2]] 2
[[1216 1075 1216 1234 1216 1095 1095 1095 1216 1223]] 1095
camIds: [[1 1 1 1 2 2 2 2 2 1]] 1
[[1216 1216 1004 1095 1223 1095 1095 1223 1223 1223]] 1095
camIds: [[2 2 2 1 1 1 1 2 1 2]] 2
[[1004 1075 1097 1017 1264 1264 1017 1189 1264 1264]] 1096
camIds: [[1 1 2 1 1 2 1 1 1 1]] 1
[[1127 1127 1220 1063 1127 1153  999 1063 1243 1178]] 1096
camIds: [[2 2 2 2 2 1 2 2 1 2]] 2
[[1097 1097 1064 1017 1017 1017 1097 1162 1017  946]] 1097
camIds: [[2 2 2 2 1 1 2 1 2 1]] 1
[[1097 1097 1017 1097 1017 1064 1181 1064 1178 1178]] 1097
camIds: [[1 1 2 1 2 2 1 2 1 1]] 2
[[1178 1017 1264 1178 1097 1017 1017 1017 1004 1178]] 1098
camIds: [[1 2 1 1 1 1 1 2 1 1]] 1
[[1068  970 1077 1077 1178 1098 1147 1068 1098 1042]] 1098
camIds: [[2 2 1 1 2 1 2 1 1 1]] 2
[[1099 1147  930 1099  999 1276 1147 1099 1199 1276]] 1099
camIds: [[2

[[1189 1189 1189 1217 1264 1217 1189 1217  946  946]] 1189
camIds: [[2 2 2 1 1 1 2 1 2 2]] 1
[[ 946 1189 1217  916  946 1007  946 1189 1217 1069]] 1189
camIds: [[2 1 1 1 2 1 2 1 1 2]] 2
[[1190 1190 1190 1190  929 1061 1091 1061 1061 1030]] 1190
camIds: [[2 2 2 2 2 2 1 2 2 1]] 1
[[1326 1422 1091 1061 1121 1068 1002 1313  872  100]] 1190
camIds: [[1 1 2 2 2 2 2 1 2 2]] 2
[[1193 1193 1193 1193  939 1225  884  967 1225  888]] 1193
camIds: [[2 2 2 2 2 2 2 2 2 2]] 1
[[ 884 1193 1193 1193 1193  884  884  884  884  967]] 1193
camIds: [[2 1 1 1 1 2 2 2 1 2]] 2
[[1123  946 1155 1221  925 1189 1037  925 1123  925]] 1195
camIds: [[1 2 2 1 2 2 1 1 1 1]] 1
[[1037 1257  968 1010  968 1217 1037 1125 1075 1127]] 1195
camIds: [[1 1 1 2 1 1 1 2 1 1]] 2
[[1197  929  929 1224 1224 1276  892  929  929 1019]] 1197
camIds: [[2 2 1 1 1 2 1 2 2 1]] 1
[[1002 1002  881 1145 1145 1197  965 1197 1224 1243]] 1197
camIds: [[2 2 1 1 1 1 1 1 2 1]] 2
[[ 970 1098 1128 1098 1063  999 1147  970 1064  970]] 1199
camIds: [[2

[[1293 1293 1293 1293  802  802 1300  802  802  802]] 1293
camIds: [[2 2 2 2 2 1 2 2 2 1]] 1
[[1293 1293  802  802 1300  488  802  320  802  802]] 1293
camIds: [[1 1 2 1 2 1 2 2 2 1]] 2
[[1326 1359 1326 1359 1359 1359 1331  100 1298 1313]] 1298
camIds: [[1 1 1 1 1 1 1 2 2 1]] 1
[[1298  882 1298 1298 1298 1359  882  881  834  882]] 1298
camIds: [[1 1 1 1 1 1 1 1 2 1]] 2
[[1342 1300  242 1342 1359  840  827 1293 1293  320]] 1300
camIds: [[1 2 2 2 1 1 1 1 2 2]] 1
[[ 645  645  196  645  196 1293 1293  600  600  101]] 1300
camIds: [[2 2 1 2 1 2 2 1 1 1]] 2
[[1301 1301 1301 1301  267  559 1350 1435  267 1326]] 1301
camIds: [[2 2 2 2 1 1 1 2 1 2]] 1
[[1301 1301  267  267  559 1350  452  452  559   99]] 1301
camIds: [[1 1 1 1 1 1 1 1 1 2]] 2
[[ 858 1178 1178  239 1285  817  239  635 1178  817]] 1305
camIds: [[1 1 1 1 1 1 1 1 1 1]] 1
[[1305   30  585  585 1342  585  585  822   30  585]] 1305
camIds: [[1 1 2 2 1 2 2 1 1 1]] 2
[[1307 1307  259  259  259  202 1307  730  730 1108]] 1307
camIds: [[2

[[1369 1392 1392 1369  739  739  739 1359  678 1369]] 1406
camIds: [[1 1 1 1 1 1 1 2 2 1]] 1
[[ 614 1392 1112 1112 1162  614  882  946 1112  614]] 1406
camIds: [[1 2 2 2 2 2 1 1 2 1]] 2
[[1407  112  131  112  112 1407  593  230  131  374]] 1407
camIds: [[2 2 1 2 2 2 1 1 2 2]] 1
[[1068 1064 1372 1128 1068 1049 1049 1068 1049 1068]] 1407
camIds: [[2 2 2 2 1 2 2 1 2 2]] 2
[[ 879  865  879  998  865 1412 1412 1009  865 1139]] 1412
camIds: [[2 2 2 1 2 2 2 2 1 1]] 1
[[1412 1142 1259 1233  704  704  939  802 1415  704]] 1412
camIds: [[1 2 2 2 1 1 1 1 2 1]] 2
[[ 328  332  827  869  332  827  569  827  328 1241]] 1415
camIds: [[2 1 1 2 1 1 1 1 2 2]] 1
[[1116  887 1116  887 1233 1233  860  269 1412  887]] 1415
camIds: [[1 2 1 2 2 2 2 2 2 2]] 2
[[1461 1461 1461 1461 1418 1461 1461 1418 1461 1461]] 1418
camIds: [[2 2 2 1 2 2 1 2 1 1]] 1
[[1461 1461 1418 1418 1461 1418 1418 1461 1461 1461]] 1418
camIds: [[1 1 1 1 2 1 1 1 2 2]] 2
[[1343  211  100  843  798 1023 1434  211  491  491]] 1421
camIds: [[1